### Fake News Classifier Using Word Embeddings and LSTM and Bidirectional LSTM

In [61]:
#importing libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
news = pd.read_csv('train_news.csv')

In [3]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
news.shape

(20800, 5)

In [6]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#dropping NAN/null values
news.dropna(inplace=True)

In [8]:
#checking if we have dropped the null values successfully.
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
#Defining x and y lables
x = news.drop('label',axis=1)
y=news['label']

In [11]:
#checking the shape of the features
[i.shape for i in (x,y)]

[(18285, 4), (18285,)]

In [16]:
#Defining Vocabulary size

voc_size= 5000

###### Data Pre-processing and One Hot Represenatation

In [17]:
msg = x.copy()

In [20]:
#We need to reset the indices as we have sampled our initial news dataset
msg.reset_index(inplace=True)

In [24]:
#impoting libraries

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
#Data pre-pocessing

ps= PorterStemmer()
corpus=[]
for i in range(0,len(msg)):
    review= re.sub('[^a-zA-Z]',' ',msg['text'][i])
    review= review.lower()
    review= review.split()
    
    review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review= ' '.join(review)
    corpus.append(review)

In [26]:
#one-hot frepresentation

one_hot_re = [one_hot(element,voc_size) for element in corpus]

In [29]:
#one_hot_re[0]

In [31]:
#using pad_sequences to make all the representations of fixed length
sentence_length=100
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)
#adding 0s before the numbers

In [32]:
embedded_docs[0]

array([1969,  624, 4993,  167, 4232, 3806, 4587,  851, 3614, 1330,  993,
       3135, 3917, 3816, 1892, 4459, 4271, 1330, 3016, 4993,  475, 3194,
       1560,  332, 3398, 1487, 2102, 4928, 1038,  412, 3872, 3532,  331,
       2379, 3303,  475, 2807, 4993, 1781, 1509, 1252, 4403, 2808,  726,
       2054, 3187, 2054, 1494, 4870,  215, 4146, 4092, 2397,  254, 3593,
       2321,  768, 3872, 4745, 1993, 4849, 3730, 4008, 3872, 1993, 4849,
       4403,  751, 3557,  692,  850, 2388, 3378,  172, 3071, 4997, 4627,
       2860, 3369, 1416,  517, 2768,  597,  738, 1341, 3765,  692,  563,
       3339,  953, 4554,  162, 4661, 3619, 2556, 4371, 2054, 2056, 4570,
       4661])

###### Creating Model

In [53]:
#using embedding layer with Deep learning model to learn weights and embedding wiyh model itself.
embedding_vector_features= 40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 40)           200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
x_final = np.array(embedded_docs)
y_final= np.array(y)

In [36]:
print(x_final.shape, y_final.shape)

(18285, 100) (18285,)


In [37]:
from sklearn.model_selection import train_test_split

x_train,x_test, y_train,y_test = train_test_split(x_final, y_final,test_size=0.33,random_state=42)

In [38]:
[i.shape for i in (x_train,x_test,y_train,y_test)]

[(12250, 100), (6035, 100), (12250,), (6035,)]

In [39]:
model.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 33s 147ms/step - loss: 0.3578 - accuracy: 0.8389 - val_loss: 0.2691 - val_accuracy: 0.8979
Epoch 2/10
192/192 [==============================] - 31s 160ms/step - loss: 0.1591 - accuracy: 0.9425 - val_loss: 0.2401 - val_accuracy: 0.9049
Epoch 3/10
192/192 [==============================] - 30s 157ms/step - loss: 0.1064 - accuracy: 0.9623 - val_loss: 0.2789 - val_accuracy: 0.9064
Epoch 4/10
192/192 [==============================] - 31s 163ms/step - loss: 0.0686 - accuracy: 0.9779 - val_loss: 0.2881 - val_accuracy: 0.8910
Epoch 5/10
192/192 [==============================] - 30s 157ms/step - loss: 0.0550 - accuracy: 0.9816 - val_loss: 0.3422 - val_accuracy: 0.9029
Epoch 6/10
192/192 [==============================] - 28s 145ms/step - loss: 0.0317 - accuracy: 0.9893 - val_loss: 0.4224 - val_accuracy: 0.9019
Epoch 7/10
192/192 [==============================] - 30s 156ms/step - loss: 0.0229 - accuracy: 0.9936 - val_loss: 0.3947 - val_ac

##### Performance Metrics and Accuracy

In [47]:
y_pred=model.predict_classes(x_test)
y_pred

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [48]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [49]:
confusion_matrix(y_test,y_pred)

array([[2986,  433],
       [ 275, 2341]], dtype=int64)

In [52]:
accuracy_score(y_test,y_pred)

0.8826843413421707

###### Using Dropout Layer

In [56]:
embedding_vector_features= 40
model1= Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_length))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dropout(0.2))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 40)           200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 40)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model1.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 40s 179ms/step - loss: 0.3775 - accuracy: 0.8267 - val_loss: 0.2527 - val_accuracy: 0.8978
Epoch 2/10
192/192 [==============================] - 27s 141ms/step - loss: 0.1808 - accuracy: 0.9340 - val_loss: 0.2499 - val_accuracy: 0.9021
Epoch 3/10
192/192 [==============================] - 26s 138ms/step - loss: 0.1238 - accuracy: 0.9558 - val_loss: 0.2555 - val_accuracy: 0.9034
Epoch 4/10
192/192 [==============================] - 34s 179ms/step - loss: 0.0955 - accuracy: 0.9687 - val_loss: 0.3025 - val_accuracy: 0.9054
Epoch 5/10
192/192 [==============================] - 36s 186ms/step - loss: 0.0756 - accuracy: 0.9749 - val_loss: 0.3050 - val_accuracy: 0.9052
Epoch 6/10
192/192 [==============================] - 44s 232ms/step - loss: 0.0567 - accuracy: 0.9827 - val_loss: 0.3676 - val_accuracy: 0.9052
Epoch 7/10
192/192 [==============================] - 39s 204ms/step - loss: 0.0462 - accuracy: 0.9855 - val_loss: 0.3501 - val_ac

In [60]:
scores = model1.evaluate(x_test, y_test, verbose=0)
print('Model score with Droput is : ',scores[1])

Model score with Droput is :  0.9009113311767578


#### Bidirectional LSTM


In [62]:
embedding_vector_features= 40
model_bi= Sequential()
model_bi.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_length))
model_bi.add(Bidirectional(LSTM(100)))
model_bi.add(Dense(1,activation='sigmoid'))
model_bi.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model_bi.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 40)           200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
model_bi.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 82s 379ms/step - loss: 0.3651 - accuracy: 0.8286 - val_loss: 0.2552 - val_accuracy: 0.8978
Epoch 2/10
192/192 [==============================] - 73s 380ms/step - loss: 0.1639 - accuracy: 0.9398 - val_loss: 0.2429 - val_accuracy: 0.9051
Epoch 3/10
192/192 [==============================] - 67s 347ms/step - loss: 0.1087 - accuracy: 0.9622 - val_loss: 0.2471 - val_accuracy: 0.9034
Epoch 4/10
192/192 [==============================] - 76s 395ms/step - loss: 0.0696 - accuracy: 0.9784 - val_loss: 0.2608 - val_accuracy: 0.8974
Epoch 5/10
192/192 [==============================] - 50s 260ms/step - loss: 0.0464 - accuracy: 0.9851 - val_loss: 0.3479 - val_accuracy: 0.9056
Epoch 6/10
192/192 [==============================] - 65s 341ms/step - loss: 0.0274 - accuracy: 0.9927 - val_loss: 0.4117 - val_accuracy: 0.9065
Epoch 7/10
192/192 [==============================] - 65s 337ms/step - loss: 0.0168 - accuracy: 0.9942 - val_loss: 0.4681 - val_ac

In [64]:
scores = model_bi.evaluate(x_test, y_test, verbose=0)
print('Model score using Bi-directional LSTM : ',scores[1])

Model score using Bi-directional LSTM :  0.8971002697944641


We see that we got a good accuracy with LSTM with Dropout and it out-performed Bidirectional LSTM in this use-case.